# Vendor Catalog Bootstrap (Minimum Seed + Admin Grant)

This notebook seeds the minimum runtime data needed for access control and grants `vendor_admin` to the current Databricks user.

## What it does
- Ensures baseline roles exist in `sec_role_definition`
- Seeds minimal role permissions in `sec_role_permission`
- Upserts current user into `app_employee_directory` and `app_user_directory`
- Grants `vendor_admin` in `sec_user_role_map`
- Adds a minimal org scope in `sec_user_org_scope`

## Before running
- Update `CATALOG_NAME` and `SCHEMA_NAME` in Cell 2 if needed.
- Run cells top to bottom once.

In [ ]:
CATALOG_NAME = "a1_dlk"
SCHEMA_NAME = "twvendor"

print(f"Catalog: {CATALOG_NAME}")
print(f"Schema: {SCHEMA_NAME}")
print("Use these values in SQL cells below if you customize this notebook.")

In [ ]:
%sql
USE CATALOG a1_dlk;
USE SCHEMA twvendor;

MERGE INTO sec_role_definition AS target
USING (
  SELECT * FROM VALUES
    ('vendor_admin',   'Admin',   'Full administrative access across workflows and governed updates.', 3, true,  true,  true,  true),
    ('vendor_steward', 'Steward', 'Data steward with elevated review/apply rights.',                 2, true,  true,  true,  true),
    ('vendor_editor',  'Editor',  'Contributor role for day-to-day edits and submissions.',        1, true,  true,  false, true),
    ('vendor_viewer',  'Viewer',  'Read-only access to vendor inventory and metadata.',            0, false, false, false, true),
    ('vendor_auditor', 'Auditor', 'Read/report access for governance and audit functions.',        0, false, true,  false, true)
) AS src(role_code, role_name, description, approval_level, can_edit, can_report, can_direct_apply, active_flag)
ON target.role_code = src.role_code
WHEN MATCHED THEN UPDATE SET
  role_name = src.role_name,
  description = src.description,
  approval_level = src.approval_level,
  can_edit = src.can_edit,
  can_report = src.can_report,
  can_direct_apply = src.can_direct_apply,
  active_flag = src.active_flag,
  updated_at = current_timestamp(),
  updated_by = lower(current_user())
WHEN NOT MATCHED THEN INSERT (
  role_code, role_name, description, approval_level, can_edit, can_report, can_direct_apply, active_flag, updated_at, updated_by
) VALUES (
  src.role_code, src.role_name, src.description, src.approval_level, src.can_edit, src.can_report, src.can_direct_apply, src.active_flag, current_timestamp(), lower(current_user())
);

In [ ]:
%sql
USE CATALOG a1_dlk;
USE SCHEMA twvendor;

MERGE INTO sec_role_permission AS target
USING (
  SELECT * FROM VALUES
    ('vendor_admin',   'vendors',   'read',  true),
    ('vendor_admin',   'vendors',   'write', true),
    ('vendor_admin',   'projects',  'read',  true),
    ('vendor_admin',   'projects',  'write', true),
    ('vendor_admin',   'admin',     'manage',true),
    ('vendor_steward', 'vendors',   'read',  true),
    ('vendor_steward', 'vendors',   'write', true),
    ('vendor_steward', 'projects',  'read',  true),
    ('vendor_steward', 'projects',  'write', true),
    ('vendor_editor',  'vendors',   'read',  true),
    ('vendor_editor',  'vendors',   'write', true),
    ('vendor_editor',  'projects',  'read',  true),
    ('vendor_editor',  'projects',  'write', true),
    ('vendor_viewer',  'vendors',   'read',  true),
    ('vendor_viewer',  'projects',  'read',  true),
    ('vendor_auditor', 'vendors',   'read',  true),
    ('vendor_auditor', 'projects',  'read',  true)
) AS src(role_code, object_name, action_code, active_flag)
ON target.role_code = src.role_code
  AND target.object_name = src.object_name
  AND target.action_code = src.action_code
WHEN MATCHED THEN UPDATE SET
  active_flag = src.active_flag,
  updated_at = current_timestamp()
WHEN NOT MATCHED THEN INSERT (
  role_code, object_name, action_code, active_flag, updated_at
) VALUES (
  src.role_code, src.object_name, src.action_code, src.active_flag, current_timestamp()
);

In [ ]:
%sql
USE CATALOG a1_dlk;
USE SCHEMA twvendor;

CREATE OR REPLACE TEMP VIEW _bootstrap_user AS
SELECT
  lower(current_user()) AS login_identifier,
  lower(current_user()) AS email,
  lower(regexp_extract(current_user(), '^([^@]+)', 1)) AS network_id,
  concat('BOOTSTRAP-', upper(regexp_extract(current_user(), '^([^@]+)', 1))) AS employee_id,
  initcap(replace(regexp_extract(current_user(), '^([^@]+)', 1), '.', ' ')) AS display_name,
  split(initcap(replace(regexp_extract(current_user(), '^([^@]+)', 1), '.', ' ')), ' ')[0] AS first_name,
  CASE
    WHEN size(split(initcap(replace(regexp_extract(current_user(), '^([^@]+)', 1), '.', ' ')), ' ')) > 1
    THEN concat_ws(' ', slice(split(initcap(replace(regexp_extract(current_user(), '^([^@]+)', 1), '.', ' ')), ' '), 2, size(split(initcap(replace(regexp_extract(current_user(), '^([^@]+)', 1), '.', ' ')), ' '))))
    ELSE 'User'
  END AS last_name;

MERGE INTO app_employee_directory AS target
USING _bootstrap_user AS src
ON lower(target.login_identifier) = src.login_identifier
WHEN MATCHED THEN UPDATE SET
  email = src.email,
  network_id = src.network_id,
  employee_id = src.employee_id,
  first_name = src.first_name,
  last_name = src.last_name,
  display_name = src.display_name,
  active_flag = 1
WHEN NOT MATCHED THEN INSERT (
  login_identifier, email, network_id, employee_id, manager_id, first_name, last_name, display_name, active_flag
) VALUES (
  src.login_identifier, src.email, src.network_id, src.employee_id, NULL, src.first_name, src.last_name, src.display_name, 1
);

MERGE INTO app_user_directory AS target
USING (
  SELECT
    concat('usr-', substr(md5(login_identifier), 1, 12)) AS user_id,
    login_identifier, email, network_id, employee_id, first_name, last_name, display_name
  FROM _bootstrap_user
) AS src
ON lower(target.login_identifier) = src.login_identifier
WHEN MATCHED THEN UPDATE SET
  email = src.email,
  network_id = src.network_id,
  employee_id = src.employee_id,
  first_name = src.first_name,
  last_name = src.last_name,
  display_name = src.display_name,
  active_flag = true,
  updated_at = current_timestamp(),
  last_seen_at = current_timestamp()
WHEN NOT MATCHED THEN INSERT (
  user_id, login_identifier, email, network_id, employee_id, manager_id, first_name, last_name, display_name, active_flag, created_at, updated_at, last_seen_at
) VALUES (
  src.user_id, src.login_identifier, src.email, src.network_id, src.employee_id, NULL, src.first_name, src.last_name, src.display_name, true, current_timestamp(), current_timestamp(), current_timestamp()
);

In [ ]:
%sql
USE CATALOG a1_dlk;
USE SCHEMA twvendor;

MERGE INTO sec_user_role_map AS target
USING (
  SELECT
    u.user_id AS user_principal,
    'vendor_admin' AS role_code,
    true AS active_flag,
    lower(current_user()) AS granted_by,
    current_timestamp() AS granted_at,
    CAST(NULL AS timestamp) AS revoked_at
  FROM app_user_directory u
  WHERE lower(u.login_identifier) = lower(current_user())
) AS src
ON target.user_principal = src.user_principal
  AND target.role_code = src.role_code
WHEN MATCHED THEN UPDATE SET
  active_flag = true,
  granted_by = src.granted_by,
  granted_at = src.granted_at,
  revoked_at = NULL
WHEN NOT MATCHED THEN INSERT (
  user_principal, role_code, active_flag, granted_by, granted_at, revoked_at
) VALUES (
  src.user_principal, src.role_code, src.active_flag, src.granted_by, src.granted_at, src.revoked_at
);

MERGE INTO sec_user_org_scope AS target
USING (
  SELECT
    u.user_id AS user_principal,
    'IT-ENT' AS org_id,
    'full' AS scope_level,
    true AS active_flag,
    current_timestamp() AS granted_at
  FROM app_user_directory u
  WHERE lower(u.login_identifier) = lower(current_user())
) AS src
ON target.user_principal = src.user_principal
  AND target.org_id = src.org_id
WHEN MATCHED THEN UPDATE SET
  scope_level = src.scope_level,
  active_flag = src.active_flag,
  granted_at = src.granted_at
WHEN NOT MATCHED THEN INSERT (
  user_principal, org_id, scope_level, active_flag, granted_at
) VALUES (
  src.user_principal, src.org_id, src.scope_level, src.active_flag, src.granted_at
);

In [ ]:
%sql
USE CATALOG a1_dlk;
USE SCHEMA twvendor;

SELECT
  u.login_identifier,
  u.user_id,
  r.role_code,
  r.active_flag,
  r.granted_at,
  r.granted_by
FROM app_user_directory u
INNER JOIN sec_user_role_map r
  ON r.user_principal = u.user_id
WHERE lower(u.login_identifier) = lower(current_user())
ORDER BY r.granted_at DESC;

SELECT role_code, role_name, active_flag
FROM sec_role_definition
ORDER BY role_code;